In [69]:
from lxml import html,cssselect
import requests
import sys

import pandas as pd

In [70]:
# set up a function to extract warrant filter results
# search by (1,warrant) (2,underlying) (3,index)
def hsi_results(option):
    option_type = str(option)
    page = requests.get('http://bank.hangseng.com/1/PA_1_1_P1/ComSvlet_MiniSite_eng_gif?app=EinvcWarrantTechSearRes&Sortby=&Sortdir=&Start=&End=&WarrCode=&UStockCode=&Searchby=3&UIndexCode=2000005001&ASType='+ option_type+'&ExpMonFrom=0&ExpYrFrom=0&ExpMonTo=0&ExpYrTo=0&OSQFrom=-1&OSQTo=-1&ImpVolFrom=0&ImpVolTo=0&PremFrom=0&PremTo=0&Issuer=0&Eval=0')
    tree = html.fromstring(page.content)
    path = '//*[@class="LightGrey"]//text()'
    path2 = '//*//tr[contains(@bgcolor, "FFFFFF")]//text()'
    quote = tree.xpath(path)
    quote2 = tree.xpath(path2)
    string = list()
    string2 = list()
    counter = 50
    db = pd.DataFrame()
    for foo in quote:
        if len(foo)==1:
            pass
        else:
            string.append(foo)
    for foo in quote2:
        if len(foo)==1:
            pass
        else:
            string2.append(foo)
    ticker, name, underlying,option_type, exercise_price ,actual_price ,price ,volume ,expiry_date = [],[],[],[],[],[],[],[],[]
    for i in range(counter):
        ticker.append(string[i*14])
        name.append(string[i*14+1])
        underlying.append(string[i*14+2])
        option_type.append(string[i*14+3])
        exercise_price.append(int(((string[i*14+4]).replace(",",""))[:-3]))
        actual_price.append(string[i*14+5])
        price.append(string[i*14+6])
        volume.append(string[i*14+7])
        expiry_date.append(string[i*14+8])
    for i in range(counter):
        ticker.append(string2[i*14])
        name.append(string2[i*14+1])
        underlying.append(string2[i*14+2])
        option_type.append(string2[i*14+3])
        exercise_price.append(int(((string2[i*14+4]).replace(",",""))[:-3]))
        actual_price.append(string2[i*14+5])
        price.append(string2[i*14+6])
        volume.append(string2[i*14+7])
        expiry_date.append(string2[i*14+8])
    db['Ticker'] = ticker
    db['Name'] = name
    db['Underlying'] = underlying
    db['Option type'] = option_type
    db['Exercise price'] = exercise_price
    db['Actual price'] = actual_price
    db['Price'] = price
    db['Volume'] = volume
    db['Expiry date'] = expiry_date
    return db

In [73]:
df_call = hsi_results(1) 
df_put = hsi_results(2)
df1 = df_call.append(df_put)
dfa = df1.sort_values(by=['Exercise price','Expiry date'],ascending=[True,True])

In [74]:
dfa.loc[dfa['Exercise price'] == 20000]

,Ticker,Name,Underlying,Option type,Exercise price,Actual price,Price,Volume,Expiry date
36,11321,SG-HSI @EP1605A,HSI,Put,20000,"20,397.11",0.029,---,2016/05/30
96,26191,GS-HSI @EP1605F,HSI,Put,20000,"20,397.11",0.027,---,2016/05/30
22,24315,SG-HSI @EC1606F,HSI,Call,20000,"20,397.11",0.060,67.5K,2016/06/29
70,11153,UB-HSI @EC1606A,HSI,Call,20000,"20,397.11",0.057,118.75K,2016/06/29
77,25618,JP-HSI @EC1606F,HSI,Call,20000,"20,397.11",0.062,32.91K,2016/06/29
79,25473,GS-HSI @EC1606F,HSI,Call,20000,"20,397.11",0.062,24.4K,2016/06/29
11,22822,UB-HSI @EP1606F,HSI,Put,20000,"20,397.11",0.063,"1,319.4K",2016/06/29
12,26014,JP-HSI @EP1606A,HSI,Put,20000,"20,397.11",0.063,977.59K,2016/06/29
55,24684,GS-HSI @EP1606C,HSI,Put,20000,"20,397.11",0.071,"11,807.25K",2016/06/29
13,27360,SG-HSI @EP1610C,HSI,Put,20000,"20,397.11",0.160,753.32K,2016/10/28
